In [25]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import os
import glob
import re

In [ ]:
'''
Current issues:
Code is not robust enough to consider all possible file names
The file names MUST end in a number before the extension.

File name structure must follow:
[FILE_NAME][NUMBER].[EXTENSION]
'''

# Extract Data from HTML Websites

In [26]:
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

In [27]:
def extract_data(folder, output_file, limit = None):
    all_links = []
    all_titles = []
    all_domains = []
    file_name = []
    total_entries = 0

    output_file = output_file + folder.split('/')[-1] + '.xlsx'

    for html_file in sorted(os.listdir(folder), key = natural_sort_key):
        if html_file.endswith('.html'):
            with open(os.path.join(folder, html_file), 'r', encoding='utf-8') as file:
                content = file.read()

            soup = BeautifulSoup(content, 'html.parser')

            # All links fall under the 'a' class
            # Extract links only that have an h3
            #This removes the links that are not search links but may appear as bubbles on google
            links_h3 = []

            for link in soup.find_all('a', href=True):
                href = link['href']
                if href.startswith('http'):
                    if link.find('h3'):
                        links_h3.append(link['href'])

            # Extract titles
            titles = []
            for title in soup.find_all('h3'):
                titles.append(title.get_text())

            # Extract domains
            domains = []
            for link in links_h3:
                domains.append(urlparse(link).netloc)

            #Make all lists the same length by filling in missing values with "NA"
            max_length = max(len(links_h3), len(titles), len(domains))

            #Fill shorter lists with "NA"
            links_h3 += ['NA'] * (max_length - len(links_h3))
            titles += ['NA'] * (max_length - len(titles))
            domains += ['NA'] * (max_length - len(domains))

            #Add a max limit
            for i in range(max_length):
              if limit and total_entries >= limit:
                break

              # Append the extracted data to the overall lists
              all_links.append(links_h3[i])
              all_titles.append(titles[i])
              all_domains.append(domains[i])
              file_name.append(html_file)
              total_entries = total_entries + 1

            if limit and total_entries >= limit:
              break



    #Create a DataFrame
    df = pd.DataFrame({
       'Links': all_links,
       'Titles': all_titles,
       'Domains': all_domains,
       'FileName': file_name
    })

    # Save to CSV or display the DataFrame

    df.to_excel(output_file, index=False)
    print(f"Data saved to {output_file}")
    print(f"len(all_links): {len(all_links)}")
    print(f"len(all_titles): {len(all_titles)}")
    print(f"len(all_domains): {len(all_domains)}")
    print(df.head())

    return all_links, all_titles, all_domains


def compare_links(all_links_parent, all_links_new, viewLinks=False, export=False, export_name='new_links.xlsx'):
    difference_list = []
    for link in all_links_new:
        if link not in all_links_parent:
            difference_list.append(link)

    if len(difference_list) == 0:
        print("No new links found")
    else:
        if viewLinks:
            for link in difference_list:
                print(link)
        print(f"New links found: {len(difference_list)}")

        if export:
            if export_name.endswith('.txt'):
                with open(export_name, 'w') as f:
                    for link in difference_list:
                        f.write(link + '\n')
                print(f"New links exported to {export_name}")

            elif export_name.endswith('.xlsx'):
                df_new_links = pd.DataFrame({'New Links': difference_list})
                df_new_links.to_excel(export_name, index=False)
                print(f"New links exported to {export_name}")

            else:
                export_name += '.xlsx'
                df_new_links = pd.DataFrame({'New Links': difference_list})
                df_new_links.to_excel(export_name, index=False)
                print(f"New links exported to {export_name}")

    return difference_list

In [28]:
def HTML_PIPELINE(parent_folder, output_file, compare_folder=None, viewLinks=False, export=False, export_name="new_links.xlsx"):

    all_links_parent, all_titles_parent, all_domains_parent = extract_data(parent_folder, output_file)

    if compare_folder:
        all_links_new, all_titles_new, all_domains_new = extract_data(compare_folder, output_file)
        difference_list = compare_links(all_links_parent, all_links_new, viewLinks=viewLinks)

        if export and difference_list:
            if not export_name.endswith('.txt') and not export_name.endswith('.xlsx'):
                export_name += '.xlsx'

            if export_name.endswith('.txt'):
                with open(export_name, 'w') as f:
                    for link in difference_list:
                        f.write(link + '\n')
                print(f"New links exported to {export_name}")

            elif export_name.endswith('.xlsx'):
                df_new_links = pd.DataFrame({'New Links': difference_list})
                df_new_links.to_excel(export_name, index=False)
                print(f"New links exported to {export_name}")
    else:
      print("Done")




## Usage Example:

HTML_PIPELINE(parent_folder, output_file, compare_folder, viewLinks=True, export=True, export_name="new_links.xlsx")


*   arg[0] = parent_folder #Mandatory | Folder containing all HTML files
*   arg[1] = output_file #Mandatory | Name of the output file
*   arg[2] = compare_folder #Optional | Compares new links to old links
*   arg[3] = viewLinks #Optional | Prints out all new links in terminal
*   arg[4] = export #Optional | Exports new links to a file
*   arg[5] = export_name #Optional | Exports new links to a file






In [29]:
'''
Usage Example:
HTML_PIPELINE(parent_folder, output_file, compare_folder=None, viewLinks=False, export=False, export_name="new_links.xlsx")

arg[0] = parent_folder #Mandatory | Folder containing all HTML files
arg[1] = output_file #Mandatory | Name of the output file
arg[2] = compare_folder #Optional | Compares new links to old links
arg[3] = viewLinks #Optional | Prints out all new links in terminal
arg[4] = export #Optional | Exports new links to a file
arg[5] = export_name #Optional | Exports new links to a file

'''

'\nUsage Example:\nHTML_PIPELINE(parent_folder, output_file, compare_folder=None, viewLinks=False, export=False, export_name="new_links.xlsx")\n\narg[0] = parent_folder #Mandatory | Folder containing all HTML files\narg[1] = output_file #Mandatory | Name of the output file\narg[2] = compare_folder #Optional | Compares new links to old links\narg[3] = viewLinks #Optional | Prints out all new links in terminal\narg[4] = export #Optional | Exports new links to a file\narg[5] = export_name #Optional | Exports new links to a file\n\n'

In [30]:
HTML_PIPELINE("/content/HTML_files", "Google Searches_167")

Data saved to Google Searches_167HTML_files.xlsx
len(all_links): 177
len(all_titles): 177
len(all_domains): 177
                                               Links  \
0  https://www.apexcoollabs.com/products/narwhals...   
1  https://startupsmagazine.co.uk/article-portabl...   
2  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...   
3  https://www.apexcoollabs.com/industries/fire-s...   
4  https://www.stucan-solutions.com/products-page...   

                                              Titles  \
0  The Next Generation Narwhals: Palm Cooling Dev...   
1  The portable body cooling device for heat illness   
2  Use of an external-cooling device for the trea...   
3        Heat Stress Mitigation for the Fire Service   
4  CGX1 Core Temperature Cooling Device | Product...   

                    Domains                                           FileName  
0      www.apexcoollabs.com  _portable cooling device_ OR _cooling device_ ...  
1    startupsmagazine.co.uk  _portable cooling devic